In [1]:
import pandas as pd
import numpy as np
import re
import json
import datetime

def time_cleaner(a_string):
    raw = str(a_string).replace('PT', '')
    
    if 'DT' in raw:
        clean_preptime = np.nan
    elif 'H' in raw and 'M' in raw:
        time = re.findall(r'(\d{1,9})', raw)
        hour = int(time[0]) * 60
        minute = int(time[1])
        clean_preptime = hour + minute
    elif 'H' in raw:
        clean_preptime = int(raw.replace('H', '')) * 60
    elif 'M' in raw:
        clean_preptime = raw.replace('M', '')
    else:
        clean_preptime = np.nan

    return clean_preptime

def date_cleaner(a_string):
    if a_string == None:
        return np.nan
    return re.sub('T.*', '', str(a_string))

def ingredient_clean(ingredient_string):
    regex = re.compile(r'½|⅓|¼|\(|\)|(\d-\d) |\d|,|\/|(oz)|(cup)|(teaspoons)|(teaspoon)|(tablespoons)|(tablespoon)|(whole)|(chopped)|( and )|(ground)|( or )|(ounce)|(weight)|( to )|( can )|( of )|(tbsp)', re.I)
    ingredient_string = regex.sub('', ingredient_string)
    return ingredient_string.replace('\n','').lower()

In [2]:
cleaner = []
with open('testing.json') as data_file:
    raw_file = data_file.readlines()
    count = 1
    for x in raw_file:
        fixed = re.sub(r'( { "\$oid" : "(.*?)(" }))', str(count), x)
        fixed = re.sub(r'("ts" : { "\$date" : (\d{1,}))( }, )', '', fixed)
        count += 1
        cleaner.append(json.loads(fixed))
raw_recipes = pd.DataFrame(cleaner)

In [3]:
raw_recipes['cookTime'] = [time_cleaner(count) for count in raw_recipes['cookTime']]
raw_recipes['totalTime'] = [time_cleaner(count) for count in raw_recipes['totalTime']]
raw_recipes['prepTime'] = [time_cleaner(count) for count in raw_recipes['prepTime']]
raw_recipes['datePublished']= [date_cleaner(date) for date in raw_recipes['datePublished']]
raw_recipes['ingredients'] = [ingredient_clean(ingredient) for ingredient in raw_recipes['ingredients']]
raw_recipes = raw_recipes.drop(['creator', 'dateModified'], 1)
# For full file, add 'recipeInstructions' to above drop list

In [4]:
from collections import Counter
import matplotlib.pyplot as plt

giant_string = ''
for ingredient in raw_recipes['ingredients']:
    giant_string = giant_string + ingredient

giant_string = giant_string.split(' ')
word_counts = dict(Counter(giant_string))
raw_recipes.head(2)

,_id,cookTime,datePublished,description,image,ingredients,name,prepTime,recipeCategory,recipeYield,source,totalTime,url
0,1,30,2013-03-11,"Late Saturday afternoon, after Marlboro Man ha...",http://static.thepioneerwoman.com/cooking/file...,biscuits s all-purpose flour baking powder s...,Drop Biscuits and Sausage Gravy,10,NaN,12,thepioneerwoman,NaN,http://thepioneerwoman.com/cooking/2013/03/dro...
1,2,20,2013-03-13,"When I was growing up, I participated in my Ep...",http://static.thepioneerwoman.com/cooking/file...,dinner rollssmall sandwich buns i used whea...,Hot Roast Beef Sandwiches,20,NaN,12,thepioneerwoman,NaN,http://thepioneerwoman.com/cooking/2013/03/hot...


In [5]:
fc_indr_dict = pd.read_csv('comp_info.tsv')

CParserError: Error tokenizing data. C error: Expected 1 fields in line 17, saw 3
